## Initialisation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import xtrack as xt
import xcoll as xc

%matplotlib tk

In [2]:
# line = xt.load('machines/fccee_lcc_v105_z_scrabON_xingON.json')
env = xt.load('https://gitlab.cern.ch/acc-models/fcc/acc-models-fcc-ee/-/raw/LCC_105/lattices/z/fccee_z.json')
line = env.lines['fccee_p_ring']

gemitt_x = 0.7e-9
gemitt_y = 2e-12
nemitt_x = gemitt_x * line.particle_ref.gamma0
nemitt_y = gemitt_y * line.particle_ref.gamma0

line.configure_radiation(model=None)

tt = line.get_table()

Loading line from dict: 100%|██████████| 21288/21288 [00:01<00:00, 21030.59it/s] 


In [3]:
coll_start = 'qd6c'
coll_dogleg_start = 'dogl_coll'
coll_dogleg_end = 'dogr_coll'
coll_end = 'qd6c.0'
coll_mid = (tt.rows[coll_end].s_center[0] + tt.rows[coll_start].s_center[0]).tolist() / 2

inj_start = 'qd6j'
inj_dogleg_start = 'dogl_inj'
inj_dogleg_end = 'dogr_inj'
inj_end = 'qd6i'
inj_mid = (tt.rows[inj_end].s_center[0] + tt.rows[inj_start].s_center[0]).tolist() / 2

exp1_start = 'qd6m.6'
exp1_end = 'qd6m'  # wraps around 0
exp1_mid = (tt.rows[exp1_end].s_center[0] - line.get_length() + tt.rows[exp1_start].s_center[0]).tolist() / 2

exp2_start = 'qd6m.0'
exp2_end = 'qd6m.1'
exp2_mid = (tt.rows[exp2_end].s_center[0] + tt.rows[exp2_start].s_center[0]).tolist() / 2

exp3_start = 'qd6m.2'
exp3_end = 'qd6m.3'
exp3_mid = (tt.rows[exp3_end].s_center[0] + tt.rows[exp3_start].s_center[0]).tolist() / 2

exp4_start = 'qd6m.4'
exp4_end = 'qd6m.5'
exp4_mid = (tt.rows[exp4_end].s_center[0] + tt.rows[exp4_start].s_center[0]).tolist() / 2

## Collimator Locations

In [4]:
s_TCP_H  = 10488.25 # tt.rows['qf9c'].s_end + 0.75
s_TCS_H1 = 10704.75  # at 32 degrees
s_TCS_H2 = 11591.65  # at 148 degrees

s_TCP_V  = 10644.55 # tt.rows['qd10d'].s_end + 0.75
s_TCS_V1 = 10958.10 # tt.rows['qd12d'].s_end + 0.75  (at 40 degrees)

s_TCP_HH  = 11037.55 # at 90 degrees
s_TCS_HH1 = 11158.95 # tt.rows[coll_dogleg_start].s_start - 0.75 (at 110 degrees)
s_TCS_HH2 = 12117.45 # at 238 degrees

s_TCP_VV  = 11706.25 # tt.rows['qd12c.0'].s_end + 0.75 (at 75 degrees)
s_TCS_VV1 = 12019.80 # tt.rows['qd10c.0'].s_end + 0.75 (at 75 + 40 degrees)

# s_TCP_HP = 33823.81 # tt.rows['qf11i'].s_end + 0.75
s_TCP_HP  = 33659.20 #
# s_TCS_HP1 = 34170.10 # tt.rows['qd10i.0'].s_start - 0.75
s_TCS_HP1 = 34158.30 # At 33.56 degrees
s_TCS_HP2 = 34702.55 # tt.rows['vsep2.2'].s_end + 0.75 (at 360 - 33.56 degrees)


env.new('tcp.h.b1', xt.Marker)
env.new('tcs.h1.b1', xt.Marker)
env.new('tcs.h2.b1', xt.Marker)
env.new('tcp.v.b1', xt.Marker)
env.new('tcs.v1.b1', xt.Marker)
env.new('tcp.hh.b1', xt.Marker)
env.new('tcs.hh1.b1', xt.Marker)
env.new('tcs.hh2.b1', xt.Marker)
env.new('tcp.vv.b1', xt.Marker)
env.new('tcs.vv1.b1', xt.Marker)
env.new('tcp.hp.b1', xt.Marker)
env.new('tcs.hp1.b1', xt.Marker)
env.new('tcs.hp2.b1', xt.Marker)
line.insert([
    env.place('tcp.h.b1', at=s_TCP_H),
    env.place('tcs.h1.b1', at=s_TCS_H1),
    env.place('tcs.h2.b1', at=s_TCS_H2),
    env.place('tcp.v.b1', at=s_TCP_V),
    env.place('tcs.v1.b1', at=s_TCS_V1),
    env.place('tcp.hh.b1', at=s_TCP_HH),
    env.place('tcs.hh1.b1', at=s_TCS_HH1),
    env.place('tcs.hh2.b1', at=s_TCS_HH2),
    env.place('tcp.vv.b1', at=s_TCP_VV),
    env.place('tcs.vv1.b1', at=s_TCS_VV1),
    env.place('tcp.hp.b1', at=s_TCP_HP),
    env.place('tcs.hp1.b1', at=s_TCS_HP1),
    env.place('tcs.hp2.b1', at=s_TCS_HP2)
])

tt = line.get_table()

Slicing line: 100%|██████████| 21280/21280 [00:00<00:00, 282989.19it/s]


In [5]:
# Verify no collimators are installed in other elements

colls = [nn for nn in tt.name if nn.startswith('tc')]
for coll in colls:
    assert tt.rows[f"{coll}<<1"].element_type[0] == 'DriftSlice'
    assert tt.rows[f"{coll}>>1"].element_type[0] == 'DriftSlice'
    assert tt.rows[f"{coll}<<1"].s_end[0] - tt.rows[f"{coll}<<1"].s_start[0] > 0.75
    assert tt.rows[f"{coll}>>1"].s_end[0] - tt.rows[f"{coll}>>1"].s_start[0] > 0.75

## Plot Phase Space

In [6]:
def plot_phase_line(phase, amp, ax, color, lw=None):
    if phase == 0:
        if lw:
            ax.plot([[-amp, amp], [-amp, amp]], [[-100, -100], [100, 100]], color=color, lw=lw)
        ax.fill_betweenx([-100, 100], -amp, -50*amp, color=color, alpha=0.25)
        ax.fill_betweenx([-100, 100], amp, 50*amp, color=color, alpha=0.25)
        return
    phi = np.deg2rad(phase)  # Counter-clockwise because in normalised space
    x = np.linspace(-10*amp, 10*amp, 400)
    y1 = (amp - x * np.cos(phi)) / np.sin(phi)
    y2 = (50 * amp - x * np.cos(phi)) / np.sin(phi)
    y3 = (-amp - x * np.cos(phi)) / np.sin(phi)
    y4 = (-50 * amp - x * np.cos(phi)) / np.sin(phi)
    if lw:
        ax.plot(x, y1, color=color, lw=lw)
        ax.plot(x, y3, color=color, lw=lw)
    ax.fill_between(x, y1, y2, color=color, alpha=0.25)
    ax.fill_between(x, y3, y4, color=color, alpha=0.25)

def make_phase_space_plot(primary_amp, secondary_amp, primary_phases, secondary_phases, color, plane, filename=None):
    fig, ax = plt.subplots(figsize=(6,6))
    x = np.linspace(-primary_amp, primary_amp, 1000)
    ax.plot(x, np.sqrt(primary_amp**2 - x**2), color='gray', lw=0.35)
    ax.plot(x, -np.sqrt(primary_amp**2 - x**2), color='gray', lw=0.35)
    x = np.linspace(-secondary_amp, secondary_amp, 1000)
    ax.plot(x, np.sqrt(secondary_amp**2 - x**2), color='gray', lw=0.35)
    ax.plot(x, -np.sqrt(secondary_amp**2 - x**2), color='gray', lw=0.35)
    # primary
    for mu in primary_phases:
        plot_phase_line(mu, primary_amp, ax, color, lw=0.5)
    # secondaries
    for mu in secondary_phases:
        plot_phase_line(mu, secondary_amp, ax, color)
    ax.set_xlim(-1.5*primary_amp, 1.5*primary_amp)
    ax.set_ylim(-1.5*primary_amp, 1.5*primary_amp)
    ax.set_xlabel(rf"$\hat{{{plane}}}\; [\sigma] $")
    ax.set_ylabel(rf"$\hat{{p_{plane}}}\; [\sigma] $")

    return fig, ax

In [8]:
fig, ax = make_phase_space_plot(8, 9.5, [0, 90.02], [32.64, 147.36, 112.34, 237.36], 'tab:blue', 'x')
ax.annotate('TCP.H', xy=(5.5, 0), color='tab:blue', weight='bold')
ax.annotate('TCP.H', xy=(-7.5, 0), color='tab:blue', weight='bold')
ax.annotate('TCS.H1', xy=(8.15, 3), color='tab:blue', rotation=-90+32.64)
ax.annotate('TCS.H1', xy=(-10, -5.5), color='tab:blue', rotation=-90+32.64)
ax.annotate('TCS.H2', xy=(8.15, -5.5), color='tab:blue', rotation=90-32.64)
ax.annotate('TCS.H2', xy=(-9.75, 3.25), color='tab:blue', rotation=90-32.64)
ax.annotate('TCP.HH', xy=(-1.25, 7.25), color='tab:blue', weight='bold')
ax.annotate('TCP.HH', xy=(-1.25, -7.75), color='tab:blue', weight='bold')
ax.annotate('TCS.HH1', xy=(-5.5, 8.25), color='tab:blue', rotation=112.34-90)
ax.annotate('TCS.HH1', xy=(2.5, -9.75), color='tab:blue', rotation=112.34-90)
ax.annotate('TCS.HH2', xy=(2.5, 8.25), color='tab:blue', rotation=237.36+90)
ax.annotate('TCS.HH2', xy=(-5.5, -10.), color='tab:blue', rotation=237.36+90)
fig.tight_layout()
fig.savefig('plots/betatron_horizontal_phase_space.png', dpi=300)

In [9]:
fig, ax = make_phase_space_plot(50, 65, [0, 75.82], [39.72, 116.12], 'tab:red', 'y')
ax.annotate('TCP.V', xy=(34.5, 0), color='tab:red', weight='bold')
ax.annotate('TCP.V', xy=(-47, 0), color='tab:red', weight='bold')
ax.annotate('TCS.V1', xy=(37, 46), color='tab:red', rotation=-90+39.72)
ax.annotate('TCS.V1', xy=(-49, -60), color='tab:red', rotation=-90+39.72)
ax.annotate('TCP.VV', xy=(4, 48), color='tab:red', weight='bold', rotation=75.82-90)
ax.annotate('TCP.VV', xy=(-20, -55), color='tab:red', weight='bold', rotation=75.82-90)
ax.annotate('TCS.VV1', xy=(-49, 50.5), color='tab:red', rotation=116.12-90)
ax.annotate('TCS.VV1', xy=(12.5, -69), color='tab:red', rotation=116.12-90)
fig.tight_layout()
fig.savefig('plots/betatron_vertical_phase_space.png', dpi=300)

In [10]:
fig, ax = make_phase_space_plot(15, 18, [0], [33.56, 326.47], 'tab:green', '\delta')
ax.annotate('TCP.HP', xy=(10, 0), color='tab:green', weight='bold')
ax.annotate('TCP.HP', xy=(-14, 0), color='tab:green', weight='bold')
ax.annotate('TCS.HP1', xy=(15.15, 5.75), color='tab:green', rotation=-90+33.56)
ax.annotate('TCS.HP1', xy=(-19, -11), color='tab:green', rotation=-90+33.56)
ax.annotate('TCS.HP2', xy=(15.5, -10), color='tab:green', rotation=90-33.56)
ax.annotate('TCS.HP2', xy=(-19, 6), color='tab:green', rotation=90-33.56)
fig.tight_layout()
fig.savefig('plots/betatron_longitudinal_phase_space.png', dpi=300)

## Plot Collimation Layout

In [11]:
# Insert markers in PF and INJ region for correct phase advances

places = []
marks_coll = tt.rows[coll_start:coll_end]
drifts = marks_coll.rows[[et.startswith('Drift') for et in marks_coll.element_type]].name
drifts_ss = marks_coll.rows[[et.startswith('Drift') for et in marks_coll.element_type]].s_start
drifts_se = marks_coll.rows[[et.startswith('Drift') for et in marks_coll.element_type]].s_end

marks_inj = tt.rows[inj_start:inj_end]
drifts = np.concat([drifts, marks_inj.rows[[et.startswith('Drift') for et in marks_inj.element_type]].name])
drifts_ss = np.concat([drifts_ss, marks_inj.rows[[et.startswith('Drift') for et in marks_inj.element_type]].s_start])
drifts_se = np.concat([drifts_se, marks_inj.rows[[et.startswith('Drift') for et in marks_inj.element_type]].s_end])

for i, (name_1, name_2, s_start, s_end) in enumerate(zip(drifts[:-1], drifts[1:], drifts_ss[:-1], drifts_se[:-1])):
    for j in range(int(s_end - s_start)):
        env.new(f"mark_{i}_{j}", xt.Marker)
        places.append(env.place(f"mark_{i}_{j}", at=s_start + j + 1))

line.insert(places)

tt = line.get_table()

Slicing line: 100%|██████████| 21306/21306 [00:00<00:00, 99772.84it/s] 


In [16]:
def plot_layout(tw, betx=True, bety=True, mux=False, muy=False, dx=False, dy=False, figsize=(17,4), tcp=None):
    fig, ax = plt.subplots(figsize=figsize)

    # Lattice
    ax1 = ax.twinx()
    twplt = tw.plot('', ax=ax1)
    ax1.get_legend().remove()
    lines = twplt.lines
    labels = twplt.legends
    ax1.set_zorder(ax.get_zorder() - 1)
    ax.patch.set_visible(False)  # hide its background, so main ax background shows through

    if betx or bety:
        if betx:
            ax.plot(tw.s, tw.betx, color='black', label=r'$\beta_{x}$', lw=1.5)
        if bety:
            ax.plot(tw.s, tw.bety, color='tab:red', label=r'$\beta_{y}$', lw=1.5)
        ax.grid(False)
        ymax = 1.1*max(tw.betx.max(), tw.bety.max())
        ax.set_ylim(0, ymax)
        ax.set_ylabel(r'$\beta$ [m]')
        ax.legend(loc='upper right')
        lines_1, labels_1 = ax.get_legend_handles_labels()
        lines += lines_1
        labels += labels_1

    # Dispersion
    if dx or dy:
        ax2 = ax.twinx()
        if dx:
            ax2.plot(tw.s, tw.dx, label=r'$D_x$', c='tab:green', lw=1)
        if dy:
            ax2.plot(tw.s, tw.dy, label=r'$D_y$', c='tab:purple', lw=1)
        ax2.set_ylabel(r'$D$ [m]')
        # ax2.set_ylim(0, 380)
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        lines += lines_2
        labels += labels_2

    # Phase advance
    if mux or muy:
        ax3 = ax.twinx()
        if mux:
            muxval = tw.mux - tw.rows[tcp or 'tcp.h.b1'].mux[0]
            mask_x = muxval >= 0
            ax3.plot(tw.s[mask_x], (muxval[mask_x] % 1)*360, label=r'$\mu_x$', c='tab:blue', lw=0.35)
        if muy:
            muyval = tw.muy - tw.rows[tcp or 'tcp.v.b1'].muy[0]
            mask_y = muyval >= 0
            ax3.plot(tw.s[mask_y], (muyval[mask_y] % 1)*360, label=r'$\mu_y$', c='tab:orange', lw=0.35)
        ax3.set_ylabel(r'$\mu$ (TCP) [deg]')
        ax3.set_ylim(0, 400)
        if dx or dy:
            ax3.spines["right"].set_position(("outward", 65))  # 50 points to the right
            ax3.patch.set_visible(False)
            ax3.set_zorder(ax2.get_zorder() - 1)
        lines_2, labels_2 = ax3.get_legend_handles_labels()
        lines += lines_2
        labels += labels_2

    # Combine legends
    if (dx or dy) and (mux or muy):
        shift = 1.225
    elif dx or dy:
        shift = 1.15
    elif mux or muy:
        shift = 1.125
    else:
        shift = 1.1
    ax.legend(lines, labels, loc='upper right', bbox_to_anchor=(shift, 1.))

    return fig, ax

In [14]:
tw0 = line.twiss4d()

In [17]:
fig, ax = plot_layout(tw0.rows[coll_start:coll_end], dx=True, mux=True)

# Collimators
ax.vlines([s_TCP_H], 0, 1000, colors='tab:blue', linestyles='--')
ax.vlines([s_TCS_H1], 0, 1000, colors='tab:blue', linestyles='--')
ax.vlines([s_TCS_H2], 0, 1000, colors='tab:blue', linestyles='--')
ax.vlines([s_TCP_HH], 0, 1000, colors='tab:blue', linestyles=':')
ax.vlines([s_TCS_HH1], 0, 1000, colors='tab:blue', linestyles=':')
ax.vlines([s_TCS_HH2], 0, 1000, colors='tab:blue', linestyles=':')
ax.annotate('TCP.H', xy=(s_TCP_H - 82, 760), color='tab:blue', weight='bold')
ax.annotate('TCS.H1', xy=(s_TCS_H1 - 102, 760), color='tab:blue', weight='bold')
ax.annotate('TCS.H2', xy=(s_TCS_H2 - 102, 760), color='tab:blue', weight='bold')
ax.annotate('TCP.HH', xy=(s_TCP_HH - 98, 760), color='tab:blue', weight='bold')
ax.annotate('TCS.HH1', xy=(s_TCS_HH1 - 115, 760), color='tab:blue', weight='bold')
ax.annotate('TCS.HH2', xy=(s_TCS_HH2 - 115, 760), color='tab:blue', weight='bold')

fig.tight_layout()
fig.savefig('plots/collimators_horiz.png', dpi=300)
fig.show()

In [18]:
fig, ax = plot_layout(tw0.rows[coll_start:coll_end], dy=True, muy=True)
ax.vlines([s_TCP_V], 0, 1000, colors='tab:red', linestyles='--')
ax.vlines([s_TCS_V1], 0, 1000, colors='tab:red', linestyles='--')
ax.vlines([s_TCP_VV], 0, 1000, colors='tab:red', linestyles=':')
ax.vlines([s_TCS_VV1], 0, 1000, colors='tab:red', linestyles=':')
ax.annotate('TCP.V', xy=(s_TCP_V - 82, 740), color='tab:red', weight='bold')
ax.annotate('TCS.V1', xy=(s_TCS_V1 - 102, 740), color='tab:red', weight='bold')
ax.annotate('TCP.VV', xy=(s_TCP_VV - 98, 740), color='tab:red', weight='bold')
ax.annotate('TCS.VV1', xy=(s_TCS_VV1 - 115, 740), color='tab:red', weight='bold')

fig.tight_layout()
fig.savefig('plots/collimators_vert.png', dpi=300)

In [19]:
fig, ax = plot_layout(tw0.rows[inj_start:inj_end], dx=True, mux=True, tcp='tcp.hp.b1')
ax.vlines([s_TCP_HP], 0, 2000, colors='tab:green', linestyles='--')
ax.vlines([s_TCS_HP1], 0, 2000, colors='tab:green', linestyles='--')
ax.vlines([s_TCS_HP2], 0, 2000, colors='tab:green', linestyles='--')
ax.annotate('TCP.HP', xy=(s_TCP_HP - 98, 1560), color='tab:green', weight='bold')
ax.annotate('TCS.HP1', xy=(s_TCS_HP1 - 115, 1560), color='tab:green', weight='bold')
ax.annotate('TCS.HP2', xy=(s_TCS_HP2 - 115, 1560), color='tab:green', weight='bold')

fig.tight_layout()
fig.savefig('plots/collimators_offmom.png', dpi=300)

### Prototyping to find locations

In [8]:
tw0.plot('betx bety')

In [ ]:
this_tw = tw0.rows[inj_start:inj_end]

fig, ax = plt.subplots(figsize=(10,6))
this_tw.plot('betx bety', ax=ax)
ax2 = ax.twinx()
ax2.plot(this_tw.s, this_tw.dx, 'tab:green')
ax.vlines([s_TCP_H], -700, 700, colors='k', linestyles='--')
ax.vlines([s_TCS_H1], -700, 700, colors='k', linestyles='--')
ax.vlines([s_TCS_H2], -700, 700, colors='k', linestyles='--')
ax.vlines([s_TCP_V], -700, 700, colors='r', linestyles='--')
ax.vlines([s_TCS_V1], -700, 700, colors='r', linestyles='--')
ax.vlines([s_TCP_HH], -700, 700, colors='k', linestyles=':')
ax.vlines([s_TCS_HH1], -700, 700, colors='k', linestyles=':')
ax.vlines([s_TCS_HH2], -700, 700, colors='k', linestyles=':')
ax.vlines([s_TCP_VV], -700, 700, colors='r', linestyles=':')
ax.vlines([s_TCS_VV1], -700, 700, colors='r', linestyles=':')
ax.vlines([s_TCP_HP], -700, 700, colors='g', linestyles='--')
ax.vlines([s_TCS_HP1], -700, 700, colors='g', linestyles='--')
ax.vlines([s_TCS_HP2], -700, 700, colors='g', linestyles='--')
ax.grid(False)

ax2 = ax.twinx()
mux = this_tw.mux - this_tw.rows['tcp.hp.b1'].mux[0]
mask_x = mux >= 0
ax2.plot(this_tw.s[mask_x], (mux[mask_x] % 1)*360, label='x')
# for ss in [-150, -100, -61, 0, 50, 100, 150]:
#     mux = this_tw.mux - this_tw.rows[s_TCP_DP+ss:s_TCP_DP+ss+1:'s'].mux[0]
#     mask_x = mux >= 0
#     ax2.plot(this_tw.s[mask_x], (mux[mask_x] % 1)*360, label='x')
# ax2.hlines([32.64, 147.36, 212.64, 327.36], this_tw.s[0], this_tw.s[-1], colors='k', lw=0.25)
# ax2.hlines([39.72, 140.28, 219.72, 320.28], this_tw.s[0], this_tw.s[-1], colors='r', lw=0.25)
# ax2.hlines([75.82, 115.54, 216.1 , 295.54, 36.1 ], this_tw.s[0], this_tw.s[-1], colors='b', lw=0.25)

mu_opt = 25.84
ax2.hlines([mu_opt, 180-mu_opt, 180+mu_opt, 360-mu_opt], this_tw.s[0], this_tw.s[-1], colors='k', lw=0.25)
mu_opt = 33.56
ax2.hlines([mu_opt, 180-mu_opt, 180+mu_opt, 360-mu_opt], this_tw.s[0], this_tw.s[-1], colors='g', lw=0.25)

In [11]:
tt.rows['dogr.*'].show()

name                    s element_type isthick isreplica parent_name iscollective       s_start ...
dogr_coll         11482.3 RBend           True     False None               False       11482.3
dogr_coll.0       11491.9 RBend           True     False None               False       11491.9
dogr_inj          34768.3 RBend           True     False None               False       34768.3
dogr_inj.0        34777.9 RBend           True     False None               False       34777.9
dogr_diag         56804.7 RBend           True     False None               False       56804.7
dogr_diag.0       56814.3 RBend           True     False None               False       56814.3
dogr_rf           79670.4 RBend           True     False None               False       79670.4
dogr_rf.0           79680 RBend           True     False None               False         79680


In [ ]:
# tw0.rows[0:5000:'s'].rows['qd.*'].show()
tw0.rows[inj_start:inj_end].show()

name                        s             x            px             y            py ...
qd6j                  33008.2  -1.53854e-19  -3.08021e-20  -1.66224e-24  -1.33404e-25
di_04..0                33010  -2.15855e-19  -3.61662e-20  -1.86328e-24  -8.17883e-26
mark_28_0               33011  -2.52021e-19  -3.61662e-20  -1.94507e-24  -8.17883e-26
di_04..1                33011  -2.52021e-19  -3.61662e-20  -1.94507e-24  -8.17883e-26
mark_28_1               33012  -2.88187e-19  -3.61662e-20  -2.02686e-24  -8.17883e-26
di_04..2                33012  -2.88187e-19  -3.61662e-20  -2.02686e-24  -8.17883e-26
mark_28_2               33013  -3.24353e-19  -3.61662e-20  -2.10864e-24  -8.17883e-26
di_04..3                33013  -3.24353e-19  -3.61662e-20  -2.10864e-24  -8.17883e-26
mark_28_3               33014  -3.60519e-19  -3.61662e-20  -2.19043e-24  -8.17883e-26
di_04..4                33014  -3.60519e-19  -3.61662e-20  -2.19043e-24  -8.17883e-26
mark_28_4               33015  -3.96686e-19  -3.61

## Collimator Optics and Layout

In [ ]:
colldb = xc.CollimatorDatabase.from_yaml('FCCee_z_v105_LCC.colldb.yaml')
line.discard_tracker()
colldb.install_black_absorbers(line=line, need_apertures=False)
tt = line.get_table()

line.build_tracker()
line.collimators.assign_optics()

Slicing line: 100%|██████████| 21306/21306 [00:00<00:00, 222549.45it/s]


In [7]:
tw0 = line.twiss4d(delta0=0)

print(f"TCP.H:   {1000*line['tcp.h.b1'].jaw[0]:.2f} mm")
print(f"TCS.H1:  {1000*line['tcs.h1.b1'].jaw[0]:.2f} mm   mux: {((tw0.rows['tcs.h1.b1'].mux[0] - tw0.rows['tcp.h.b1'].mux[0]) % 1) * 360:.2f} deg")
print(f"TCS.H2:  {1000*line['tcs.h2.b1'].jaw[0]:.2f} mm   mux: {((tw0.rows['tcs.h2.b1'].mux[0] - tw0.rows['tcp.h.b1'].mux[0]) % 1) * 360:.2f} deg")
print()
print(f"TCP.V:   {1000*line['tcp.v.b1'].jaw[0]:.2f} mm")
print(f"TCS.V1:  {1000*line['tcs.v1.b1'].jaw[0]:.2f} mm   muy: {((tw0.rows['tcs.v1.b1'].muy[0] - tw0.rows['tcp.v.b1'].muy[0]) % 1) * 360:.2f} deg")
print()
print(f"TCP.HH:  {1000*line['tcp.hh.b1'].jaw[0]:.2f} mm   mux: {((tw0.rows['tcp.hh.b1'].mux[0] - tw0.rows['tcp.h.b1'].mux[0]) % 1) * 360:.2f} deg")
print(f"TCS.HH1: {1000*line['tcs.hh1.b1'].jaw[0]:.2f} mm   mux: {((tw0.rows['tcs.hh1.b1'].mux[0] - tw0.rows['tcp.h.b1'].mux[0]) % 1) * 360:.2f} deg")
print(f"TCS.HH2: {1000*line['tcs.hh2.b1'].jaw[0]:.2f} mm   mux: {((tw0.rows['tcs.hh2.b1'].mux[0] - tw0.rows['tcp.h.b1'].mux[0]) % 1) * 360:.2f} deg")
print()
print(f"TCP.VV:  {1000*line['tcp.vv.b1'].jaw[0]:.2f} mm   muy: {((tw0.rows['tcp.vv.b1'].muy[0] - tw0.rows['tcp.v.b1'].muy[0]) % 1) * 360:.2f} deg")
print(f"TCS.VV1: {1000*line['tcs.vv1.b1'].jaw[0]:.2f} mm   muy: {((tw0.rows['tcs.vv1.b1'].muy[0] - tw0.rows['tcp.v.b1'].muy[0]) % 1) * 360:.2f} deg")
print()
print(f"TCP.HP:  {1000*line['tcp.hp.b1'].jaw[0]*1000:.2f} mm    Dx: {tw0.rows['tcp.hp.b1'].dx[0]:.2f} m   delta_acc: {abs(line['tcp.hp.b1'].jaw[0] / tw0.rows['tcp.hp.b1'].dx[0]*100):.2f}%")
print(f"TCS.HP1: {1000*line['tcs.hp1.b1'].jaw[0]*1000:.2f} mm    Dx: {tw0.rows['tcs.hp1.b1'].dx[0]:.2f} m   delta_acc: {abs(line['tcs.hp1.b1'].jaw[0] / tw0.rows['tcs.hp1.b1'].dx[0]*100):.2f}%   mux: {((tw0.rows['tcs.hp1.b1'].mux[0] - tw0.rows['tcp.hp.b1'].mux[0]) % 1) * 360:.2f} deg")
print(f"TCS.HP2: {1000*line['tcs.hp2.b1'].jaw[0]*1000:.2f} mm    Dx: {tw0.rows['tcs.hp2.b1'].dx[0]:.2f} m   delta_acc: {abs(line['tcs.hp2.b1'].jaw[0] / tw0.rows['tcs.hp2.b1'].dx[0]*100):.2f}%   mux: {((tw0.rows['tcs.hp2.b1'].mux[0] - tw0.rows['tcp.hp.b1'].mux[0]) % 1) * 360:.2f} deg")

TCP.H:   5.71 mm
TCS.H1:  5.00 mm   mux: 32.64 deg
TCS.H2:  5.03 mm   mux: 147.36 deg

TCP.V:   1.91 mm
TCS.V1:  2.47 mm   muy: 40.30 deg

TCP.HH:  3.74 mm   mux: 90.00 deg
TCS.HH1: 2.45 mm   mux: 112.23 deg
TCS.HH2: 5.69 mm   mux: 237.36 deg

TCP.VV:  1.91 mm   muy: 75.82 deg
TCS.VV1: 2.47 mm   muy: 116.12 deg

TCP.HP:  8601.33 mm    Dx: -0.66 m   delta_acc: 1.30%
TCS.HP1: 8539.43 mm    Dx: -0.53 m   delta_acc: 1.61%   mux: 33.56 deg
TCS.HP2: 7727.76 mm    Dx: -0.10 m   delta_acc: 7.93%   mux: 326.44 deg


In [8]:
print(f"{np.arccos(8/9.5)*180/np.pi:.2f}")  # 8.5 10 11
print(f"{np.arccos(50/65)*180/np.pi:.2f}")  # 50 65 65
print(f"{np.arccos(15/18)*180/np.pi:.2f}")
print(f"{np.arccos(18/21.6)*180/np.pi:.2f}")

32.64
39.72
33.56
33.56


In [10]:
res = ["    Name            Type       Plane      s [m]  Material      Length [m]    nsigma    Half-gap x [mm]    Half-gap y [mm]       alfx      alfy    betx [m]    bety [m]        dx [m]"]

for i, nn in enumerate(colldb.collimator_names):
    mess = f"{i:<2}  {nn:<16}{colldb[nn]['stage'].capitalize():<11}"
    plane = 'H' if np.isclose(line[nn].angle, 0) else 'V'
    material = {'mogr': 'MoGr', 'mo': 'Mo', 'iner': 'Inermet180'}[colldb[nn]['material']]
    mess += f"{plane}{line.get_s_position(nn):>15.1f}  {material:<4} {line[nn].length:>19.2f}  {line[nn].gap:>8.1f}  "
    if plane == 'H':
        mess += f"{line[nn].jaw[0]*1000:>17.5f}"
    else:
        mess += f"               30"
    if plane == 'V':
        mess += f"{line[nn].jaw[0]*1000:>19.5f}"
    else:
        mess += f"                 30"
    mess += f"{line[nn].optics['upstream'].alfx[0]:>11.5f}"
    mess += f"{line[nn].optics['upstream'].alfy[0]:>10.5f}"
    mess += f"{line[nn].optics['upstream'].betx[0]:>12.5f}"
    mess += f"{line[nn].optics['upstream'].bety[0]:>12.5f}"
    mess += f"{line[nn].optics['upstream'].dx[0]:>14.6f}"
    res.append(mess)
#  0  tct.h.2.b1      Tertiary   H        21959.3  MoGr                0.25      10.5            2.42906                 30   0.823576  -14.6677     72.3212      2424.5   0.000807787

with open('FCCee_z_v105_LCC_collimator_table.dat', 'w') as f:
    f.write("\n".join(res))

## Plot Betatronic-Longitudinal Collimator Cuts

In [17]:
# Perform a bunch of twisses for different delta
delta_step = 0.0004
delta_max_pos = 0.0136  #  0.015
delta_max_neg = -0.0104 # -0.015

tw = {0: tw0}
beam_sizes = {0:tw[0].get_beam_covariance(gemitt_x=gemitt_x, gemitt_y=gemitt_y)}

In [18]:
def calculate_incremental_step(line, prev_delta, next_delta, prev_res, depth=0):
    print(f"{'  '*depth}Calculating twiss with delta = {round(next_delta, 12)}")
    try:
        res = line.twiss4d(delta0=next_delta, co_guess=prev_res.particle_on_co)
    except ValueError:
        if depth > 3:
            print(f"{'  '*depth}-> FAILED for delta = {round(next_delta, 12)}, giving up")
            return prev_res, True
        res = prev_res
        step = (next_delta - prev_delta)/40
        print(f"{'  '*depth}-> FAILED for delta = {round(next_delta, 12)}, trying with smaller steps ({round(step, 12)})")
        for i in range(1, 41):
            this_prev_delta = prev_delta + (i-1)*step
            this_delta = prev_delta + i*step
            res, failed = calculate_incremental_step(line, this_prev_delta, this_delta, res, depth=depth+1)
            if failed:
                return res, True
    return res, False

delta_plus = np.arange(0, delta_max_pos+delta_step, delta_step)
delta_min = np.arange(0, delta_max_neg-delta_step, -delta_step)
deltas = np.concatenate((delta_plus, delta_min))
for i, delta in enumerate(deltas):
    this_delta = round(delta,6)
    if this_delta in tw:
        res = tw[0]
        continue
    res, failed = calculate_incremental_step(line, deltas[i-1], delta, res)
    if failed:
        continue
    tw[this_delta] = res
    beam_sizes[this_delta] = tw[this_delta].get_beam_covariance(gemitt_x=gemitt_x, gemitt_y=gemitt_y)
tw = dict(sorted(tw.items()))

Calculating twiss with delta = 0.0004
Calculating twiss with delta = 0.0008
Calculating twiss with delta = 0.0012
Calculating twiss with delta = 0.0016
Calculating twiss with delta = 0.002
Calculating twiss with delta = 0.0024
Calculating twiss with delta = 0.0028
Calculating twiss with delta = 0.0032
Calculating twiss with delta = 0.0036
Calculating twiss with delta = 0.004
Calculating twiss with delta = 0.0044
Calculating twiss with delta = 0.0048
Calculating twiss with delta = 0.0052
Calculating twiss with delta = 0.0056
Calculating twiss with delta = 0.006
Calculating twiss with delta = 0.0064
Calculating twiss with delta = 0.0068
Calculating twiss with delta = 0.0072
Calculating twiss with delta = 0.0076
Calculating twiss with delta = 0.008
Calculating twiss with delta = 0.0084
Calculating twiss with delta = 0.0088
Calculating twiss with delta = 0.0092
Calculating twiss with delta = 0.0096
Calculating twiss with delta = 0.01
Calculating twiss with delta = 0.0104
Calculating twiss 

In [25]:
def add_jaw_curve_H(ax, tw, beam_sizes, name, amp, color, lw=None):
    delta = np.array(list(tw.keys()))
    jaw_L =  amp * np.sqrt(tw[0].rows[name].betx[0]*gemitt_x) + tw[0].rows[name].x[0]
    jaw_R = -amp * np.sqrt(tw[0].rows[name].betx[0]*gemitt_x) + tw[0].rows[name].x[0]
    nx_L = np.array([(jaw_L - tw[dd].rows[name].x[0]) / beam_sizes[dd].rows[name].sigma_x[0] for dd in delta])
    nx_R = np.array([(jaw_R - tw[dd].rows[name].x[0]) / beam_sizes[dd].rows[name].sigma_x[0] for dd in delta])
    ax.fill_betweenx(1000*delta, nx_L, 15*amp, color=color, alpha=0.25)
    ax.fill_betweenx(1000*delta, nx_R, -15*amp, color=color, alpha=0.25)
    if lw:
        ax.plot(nx_L, 1000*delta, label=f'{name} Left Jaw', color=color, lw=lw)
        ax.plot(nx_R, 1000*delta, label=f'{name} Right Jaw', color=color, lw=lw)

def add_jaw_curve_V(ax, tw, beam_sizes, name, amp, color, lw=None):
    delta = np.array(list(tw.keys()))
    jaw_L =  amp * np.sqrt(tw[0].rows[name].bety[0]*gemitt_y) + tw[0].rows[name].y[0]
    jaw_R = -amp * np.sqrt(tw[0].rows[name].bety[0]*gemitt_y) + tw[0].rows[name].y[0]
    ny_L = np.array([(jaw_L - tw[dd].rows[name].y[0]) / beam_sizes[dd].rows[name].sigma_y[0] for dd in delta])
    ny_R = np.array([(jaw_R - tw[dd].rows[name].y[0]) / beam_sizes[dd].rows[name].sigma_y[0] for dd in delta])
    ax.fill_betweenx(1000*delta, ny_L, 15*amp, color=color, alpha=0.25)
    ax.fill_betweenx(1000*delta, ny_R, -15*amp, color=color, alpha=0.25)
    if lw:
        ax.plot(ny_L, 1000*delta, label=f'{name} Left Jaw', color=color, lw=lw)
        ax.plot(ny_R, 1000*delta, label=f'{name} Right Jaw', color=color, lw=lw)


def plot_longitudinal_coupling(tw, beam_sizes, primary_amps, secondary_amps, primaries, secondaries, colors,
                               filename=None, figsize=(8, 6), plane='H'):
    if not plane in ['H', 'V']:
        raise ValueError("plane must be 'H' or 'V'")
    if not hasattr(colors, '__iter__') or isinstance(colors, str):
        assert not hasattr(primary_amps, '__iter__') or isinstance(primary_amps, str)
        assert not hasattr(secondary_amps, '__iter__') or isinstance(secondary_amps, str)
        colors = [colors]
        primary_amps = [primary_amps]
        secondary_amps = [secondary_amps]
        primaries = [primaries]
        secondaries = [secondaries]
    assert len(primary_amps) == len(primaries) == len(colors) == len(secondaries) == len(secondary_amps)

    _, ax = plt.subplots(figsize=figsize)
    for names, amp, color in zip(primaries, primary_amps, colors):
        if not hasattr(names, '__iter__') or isinstance(names, str):
            names = [names]
        for name in names:
            if plane == 'H':
                add_jaw_curve_H(ax, tw, beam_sizes, name, amp, color, lw=0.5)
            else:
                add_jaw_curve_V(ax, tw, beam_sizes, name, amp, color, lw=0.5)
    for names, amp, color in zip(secondaries, secondary_amps, colors):
        if not hasattr(names, '__iter__') or isinstance(names, str):
            names = [names]
        for name in names:
            if plane == 'H':
                add_jaw_curve_H(ax, tw, beam_sizes, name, amp, color, lw=0.5)
            else:
                add_jaw_curve_V(ax, tw, beam_sizes, name, amp, color, lw=0.5)

    if plane == 'H':
        ax.set_xlabel(r"$x\; [\sigma] $")
    else:
        ax.set_xlabel(r"$y\; [\sigma] $")
    ax.set_ylabel(r"$\delta\; [10^{-3}] $")
    ax.set_xlim((-1.5*min(primary_amps), 1.5*min(primary_amps)))
    ax.set_ylim(1000*min(tw.keys()), 1000*max(tw.keys()))
    ax.grid(True)
    plt.tight_layout()
    if filename:
        plt.savefig(f'plots/{filename}.png', dpi=300)
    plt.show()

In [26]:
plot_longitudinal_coupling(tw, beam_sizes, 8, 9.5, ['TCP.H', 'TCP.PH'], ['TCS.H1', 'TCS.H2', 'TCS.PH1', 'TCS.PH2'], 'tab:blue',
                           'horizontal_longitudinal_coupling')

plot_longitudinal_coupling(tw, beam_sizes, 50, 65, ['TCP.V','TCP.PV'], ['TCS.V1', 'TCS.PV1'], 'tab:red',
                           'vertical_longitudinal_coupling', plane='V')

plot_longitudinal_coupling(tw, beam_sizes, 18, 20, ['TCP.DP'], ['TCS.DP1'], 'tab:green', 'longitudinal_coupling')

plot_longitudinal_coupling(tw, beam_sizes, [8, 18], [9.5, 20], [['TCP.H', 'TCP.PH'], ['TCP.DP']],
                           [['TCS.H1', 'TCS.H2', 'TCS.PH1', 'TCS.PH2'], ['TCS.DP1']], ['tab:blue', 'tab:green'],
                           'all_longitudinal')